## Notebook to reproduce & debug `prepare_qcfc_plotting`

In [1]:
# ── Cell 0: insert repo into PATH and stub the missing bct function ──
import sys, types
from pathlib import Path

# 1) Point to your repo root so "import fmriprep_denoise" works
REPO_ROOT = Path("/home/seann/scratch/denoise/fmriprep-denoise-benchmark")
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))

# 2) Create a fake `bct` module with the expected function
bct = types.ModuleType("bct")
def modularity_louvain_und_sign(*args, **kwargs):
    # stub—only needed so the import succeeds; unused for QC‑FC
    return None
bct.modularity_louvain_und_sign = modularity_louvain_und_sign
sys.modules["bct"] = bct

# Verify imports now work:
from fmriprep_denoise.features import (
    significant_level,
    calculate_median_absolute,
    get_atlas_pairwise_distance,
)
print("✅ fmriprep_denoise.features loaded successfully!")

✅ fmriprep_denoise.features loaded successfully!


In [2]:
# Cell 1: imports for debug pipeline
import logging
from pathlib import Path

import pandas as pd
import numpy as np
from scipy.stats import spearmanr

# fmriprep_denoise imports now work thanks to Cell 0
from fmriprep_denoise.features import (
    significant_level,
    calculate_median_absolute,
    get_atlas_pairwise_distance,
)
from fmriprep_denoise.visualization.tables import group_name_rename

logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")
logger = logging.getLogger(__name__)

In [3]:
# Cell 2: user parameters
PATH_ROOT        = Path("/home/seann/scratch/denoise/fmriprep-denoise-benchmark/"
                        "outputs/denoise-metrics-atlas.5-4.27.25")
DATASETS         = ["ds000228"]
DATASET          = DATASETS[0]            # <-- add this
FMRIPREP_VERSION = "fmriprep-25.0.0"
CRITERIA         = "stringent"              # "minimal" or "stringent"
EXCLUDED         = []

# for the glob‐pattern in Cell 3
ATLAS_NAME       = "Schaefer2018"
DIMENSION        = 424   

In [4]:
# Cell 3: replicate the glob logic from utils._get_connectome_metric_paths
def _get_connectome_metric_paths(dataset, fmriprep_version, metric,
                                 atlas_name, dimension, path_root):
    atlas_name = "*" if atlas_name is None else atlas_name
    dimension  = "*" if dimension  is None else dimension
    rel = (
        f"{dataset}/{fmriprep_version}/"
        f"dataset-{dataset}_atlas-{atlas_name}_nroi-{dimension}_{metric}.tsv"
    )
    files = list(path_root.glob(rel))
    print(f"🔍 Pattern: {rel}")
    print("→ Found files:", files)
    labels = [f.name.split(f"_{metric}")[0] for f in files]
    print("→ Labels:", labels)
    return files, labels

files, labels = _get_connectome_metric_paths(
    DATASET, FMRIPREP_VERSION, "qcfc", ATLAS_NAME, DIMENSION, PATH_ROOT
)
assert files, "No QC‑FC files found; verify PATH_ROOT and version"

🔍 Pattern: ds000228/fmriprep-25.0.0/dataset-ds000228_atlas-Schaefer2018_nroi-424_qcfc.tsv
→ Found files: [PosixPath('/home/seann/scratch/denoise/fmriprep-denoise-benchmark/outputs/denoise-metrics-atlas.5-4.27.25/ds000228/fmriprep-25.0.0/dataset-ds000228_atlas-Schaefer2018_nroi-424_qcfc.tsv')]
→ Labels: ['dataset-ds000228_atlas-Schaefer2018_nroi-424']


In [5]:
# Cell 4: load a QC‑FC table and reshape its columns to a MultiIndex
def _qcfc_bygroup(metric, path_tsv):
    df = pd.read_csv(path_tsv, sep="\t", index_col=0, header=[0,1])
    df = df.rename(columns=group_name_rename)
    before_shape = df.shape
    df = df.filter(regex=metric)
    after_shape  = df.shape
    tuples = [
        (grp, strat.replace(f"_{metric}", ""))
        for grp, strat in df.columns
    ]
    df.columns = pd.MultiIndex.from_tuples(
        tuples, names=["groups","strategy"]
    )
    print(f"_qcfc_bygroup('{metric}') {before_shape} → {after_shape} columns: {df.columns}")
    return df

# Test it on p‐values
qcfc_pv_df = _qcfc_bygroup("pvalue", files[0])
qcfc_corr_df = _qcfc_bygroup("correlation", files[0])

_qcfc_bygroup('pvalue') (89676, 42) → (89676, 21) columns: MultiIndex([('full_sample',        'baseline'),
            (      'child',        'baseline'),
            (      'adult',        'baseline'),
            ('full_sample',          'simple'),
            (      'child',          'simple'),
            (      'adult',          'simple'),
            ('full_sample',      'simple+gsr'),
            (      'child',      'simple+gsr'),
            (      'adult',      'simple+gsr'),
            ('full_sample',     'scrubbing.5'),
            (      'child',     'scrubbing.5'),
            (      'adult',     'scrubbing.5'),
            ('full_sample', 'scrubbing.5+gsr'),
            (      'child', 'scrubbing.5+gsr'),
            (      'adult', 'scrubbing.5+gsr'),
            ('full_sample',         'compcor'),
            (      'child',         'compcor'),
            (      'adult',         'compcor'),
            ('full_sample',           'aroma'),
            (      'child',  

In [6]:
# Cell 5: stack the MultiIndex columns into long form
# ────────────────────────────────────────────────────────────────
# qcfc_pv_df is a DataFrame with MultiIndex columns (groups, strategy)
melted = qcfc_pv_df.stack(level=["groups","strategy"]).reset_index(name="value")

# Rename the automatically named “level_0” index column if you like:
melted = melted.rename(columns={"level_0": "edge_id"})

print("Melted shape:", melted.shape)
print(melted.head())

# Quick p‐value distribution check
print("p-value stats:\n", melted["value"].describe())

Melted shape: (1883196, 4)
   edge_id groups         strategy     value
0        0  adult         baseline  0.236793
1        0  adult           simple  0.756426
2        0  adult       simple+gsr  0.167845
3        0  adult      scrubbing.5  0.834083
4        0  adult  scrubbing.5+gsr  0.057266


/tmp/ipykernel_4191598/30838691.py:4: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  melted = qcfc_pv_df.stack(level=["groups","strategy"]).reset_index(name="value")


p-value stats:
 count    1.883196e+06
mean     1.936120e-01
std      2.189894e-01
min      2.297417e-07
25%      4.524548e-02
50%      9.642476e-02
75%      2.776672e-01
max      9.999460e-01
Name: value, dtype: float64


In [7]:
# Cell 6: apply uncorrected threshold and compute %
melted["p_sig"] = (
    melted.groupby(["groups","strategy"])["value"]
          .transform(significant_level)
)
# Inspect one strategy
grp = melted.query("groups=='full_sample' & strategy=='baseline'")
print("baseline rows:", len(grp), "sum(sig):", grp["p_sig"].sum())
print("baseline % sig:", grp["p_sig"].mean()*100)

# Summarise ALL strategies
uncorr = (
    melted.groupby(["groups","strategy"])["p_sig"]
          .mean()*100
).unstack()
print("Uncorrected % significant by strategy:\n", uncorr)

baseline rows: 89676 sum(sig): 86827
baseline % sig: 96.82300727061866
Uncorrected % significant by strategy:
 strategy        aroma   baseline    compcor  scrubbing.5  scrubbing.5+gsr  \
groups                                                                      
adult        2.333958  99.821580   0.041260     8.908738         0.079174   
child        0.086980   0.336768  92.167358     1.013649         0.255364   
full_sample  0.088095  96.823007   1.380525    92.462866         0.354610   

strategy        simple  simple+gsr  
groups                              
adult        99.441322    0.717026  
child         6.401936    0.596592  
full_sample  98.485659    1.698336  


In [8]:
# Cell 7: apply FDR correction and compute %
melted["p_fdr"] = (
    melted.groupby(["groups","strategy"])["value"]
          .transform(significant_level, correction="fdr_bh")
)
grp_fdr = melted.query("groups=='full_sample' & strategy=='baseline'")
print("baseline % FDR sig:", grp_fdr["p_fdr"].mean()*100)

fdr = (
    melted.groupby(["groups","strategy"])["p_fdr"]
          .mean()*100
).unstack()
print("FDR % significant by strategy:\n", fdr)

baseline % FDR sig: 96.58660065123334
FDR % significant by strategy:
 strategy     aroma   baseline  compcor  scrubbing.5  scrubbing.5+gsr  \
groups                                                                 
adult          0.0  99.821580  0.00000          0.0              0.0   
child          0.0   0.000000  0.00000          0.0              0.0   
full_sample    0.0  96.586601  0.00223          0.0              0.0   

strategy        simple  simple+gsr  
groups                              
adult        99.436862    0.000000  
child         0.000000    0.000000  
full_sample  98.466702    0.001115  


In [9]:
# Cell 8: compute median absolute QC‐FC per strategy
mad = qcfc_corr_df.apply(calculate_median_absolute)
print("First few median(|r|):\n", mad.head())

# Melt and summarise
mad_long = mad.reset_index().melt(
    id_vars=["groups","strategy"],
    value_name="qcfc_mad"
)
meds = mad_long.groupby(["groups","strategy"])["qcfc_mad"]\
               .median().unstack()
print("Median absolute QC‑FC by strategy:\n", meds)

First few median(|r|):
 groups       strategy
full_sample  baseline    0.182147
child        baseline    0.146612
adult        baseline    0.542621
full_sample  simple      0.207833
child        simple      0.190845
dtype: float64
Median absolute QC‑FC by strategy:
 strategy        aroma  baseline   compcor  scrubbing.5  scrubbing.5+gsr  \
groups                                                                    
adult        0.333517  0.542621  0.058185     0.339532         0.178702   
child        0.049048  0.146612  0.202070     0.172367         0.074967   
full_sample  0.057280  0.182147  0.143911     0.173501         0.096517   

strategy       simple  simple+gsr  
groups                             
adult        0.446453    0.278596  
child        0.190845    0.138672  
full_sample  0.207833    0.144414  


In [10]:
# Cell 9 (fixed): compute Spearman ρ(distance, QC‑FC) per (group,strategy)
label0 = labels[0].replace(f"dataset-{DATASET}_", "")
atlas = label0.split("atlas-")[-1].split("_")[0]
dim   = label0.split("nroi-")[-1].split("_")[0]

# use the same exclusion you validated already
excluded_rois_paths = {
    "fmriprep-25.0.0": "/home/seann/scratch/halfpipe_test/test14/"
                      "derivatives/denoise_0.8subjectthreshold/rois_dropped.csv",
    "fmriprep-20.2.7": "/home/seann/scratch/halfpipe_test/test15/"
                      "derivatives/denoise_0.8subjectthreshold/rois_dropped.csv",
}
excluded_rois_path = excluded_rois_paths[FMRIPREP_VERSION]

dist_df = get_atlas_pairwise_distance(
    atlas, dim, excluded_rois_path=excluded_rois_path
)
d = dist_df["distance"].values

# Build a list of ((group,strategy), rho) pairs
rho_list = []
for grp, strat in qcfc_corr_df.columns:
    y = qcfc_corr_df[(grp, strat)].values
    rho = spearmanr(d, y).correlation
    rho_list.append(((grp, strat), rho))

# Construct a MultiIndex Series
dist_df2 = pd.Series(
    dict(rho_list),
    name="corr_motion_distance"
)
dist_df2.index = pd.MultiIndex.from_tuples(
    dist_df2.index, names=["groups","strategy"]
)

print("✔️ Distance‐dependence per (group,strategy):")
print(dist_df2.head())

Fetching centroids for atlas: Schaefer2018 with dimension: 424
Loading centroids from precomputed TSV: /home/seann/scratch/denoise/fmriprep-denoise-benchmark/outputs/denoise-metrics-atlas.5-4.17.25/ds000228/fmriprep-25.0.0/atlas-Schaefer2018Combined_centroids.tsv
Centroids successfully loaded. Shape: (424, 3)
Computing pairwise distances using cdist...
Pairwise distance computation complete. Returning distance dataframe with shape: (89676, 3)
✔️ Distance‐dependence per (group,strategy):
groups       strategy
full_sample  baseline   -0.272920
child        baseline   -0.235973
adult        baseline   -0.108527
full_sample  simple     -0.348687
child        simple     -0.286502
Name: corr_motion_distance, dtype: float64


In [11]:
# Cell 10: Rebuild manual summary and save to TSV

import pandas as pd

# 1) Reconstruct manual summary from previous cells
sig_df    = pd.DataFrame(uncorr.stack(),    columns=["qcfc_significant"])
fdr_df    = pd.DataFrame(fdr.stack(),       columns=["qcfc_fdr_significant"])
median_df = pd.DataFrame(meds.stack(),      columns=["qcfc_mad"])
# dist_df2 from Cell 9 is a MultiIndex Series of corr_motion_distance

summary_manual = pd.concat([sig_df, fdr_df, median_df, dist_df2], axis=1)
summary_manual.index.names = ["groups", "strategy"]

print("✔️ Manual summary (first rows):\n", summary_manual.head(), "\n")

# 2) Build output path
outpath = (
    PATH_ROOT / DATASET / FMRIPREP_VERSION /
    f"{DATASET}_{FMRIPREP_VERSION.replace('.', '-')}"
    f"_desc-{CRITERIA}_summary_test.tsv"
)
outpath.parent.mkdir(parents=True, exist_ok=True)

# 3) Save the DataFrame to TSV
summary_manual.to_csv(outpath, sep="\t")
print(f"✅ Saved manual QC‑FC summary to:\n  {outpath}")

✔️ Manual summary (first rows):
                         qcfc_significant  qcfc_fdr_significant  qcfc_mad  \
groups strategy                                                            
adult  aroma                    2.333958               0.00000  0.333517   
       baseline                99.821580              99.82158  0.542621   
       compcor                  0.041260               0.00000  0.058185   
       scrubbing.5              8.908738               0.00000  0.339532   
       scrubbing.5+gsr          0.079174               0.00000  0.178702   

                        corr_motion_distance  
groups strategy                               
adult  aroma                        0.018173  
       baseline                    -0.108527  
       compcor                     -0.202154  
       scrubbing.5                 -0.118843  
       scrubbing.5+gsr             -0.193409   

✅ Saved manual QC‑FC summary to:
  /home/seann/scratch/denoise/fmriprep-denoise-benchmark/outputs/denoi

In [12]:
# Cell 10: Rebuild manual summary, load existing, align, and compare

import pandas as pd

# 1) Reconstruct manual summary
#    (Make sure uncorr, fdr, meds, dist_df2 are still in memory from Cells 6–9)
sig_df    = pd.DataFrame(uncorr.stack(),    columns=["qcfc_significant"])
fdr_df    = pd.DataFrame(fdr.stack(),       columns=["qcfc_fdr_significant"])
median_df = pd.DataFrame(meds.stack(),      columns=["qcfc_mad"])
# dist_df2 from fixed Cell 9 is already a MultiIndex Series

summary_manual = pd.concat([sig_df, fdr_df, median_df, dist_df2], axis=1)
summary_manual.index.names = ["groups", "strategy"]

print("✔️ Manual summary:\n", summary_manual.head(), "\n")

# 2) Load existing summary TSV with two‐row header
summary_path = (
    PATH_ROOT / DATASET / FMRIPREP_VERSION /
    f"{DATASET}_{FMRIPREP_VERSION.replace('.', '-')}"
    f"_desc-{CRITERIA}_summary.tsv"
)
existing = pd.read_csv(
    summary_path,
    sep="\t",
    header=[0,1],     # two‐row header for the columns
    index_col=[0,1]   # two‐column index
)

# 3) Subset to QC‑FC metrics and drop second‐level labels
qcfc_metrics = ["qcfc_significant","qcfc_fdr_significant","qcfc_mad","corr_motion_distance"]
existing_qcfc = existing[qcfc_metrics]
existing_qcfc.columns = existing_qcfc.columns.droplevel(1)

# 4) Align index names
existing_qcfc.index.names = ["groups","strategy"]

print("✔️ Existing QC‑FC summary:\n", existing_qcfc.head(), "\n")

# 5) Compare
pd.testing.assert_frame_equal(summary_manual, existing_qcfc)
print("✅ Manual QC‑FC summary MATCHES the existing file exactly.")

✔️ Manual summary:
                         qcfc_significant  qcfc_fdr_significant  qcfc_mad  \
groups strategy                                                            
adult  aroma                    2.333958               0.00000  0.333517   
       baseline                99.821580              99.82158  0.542621   
       compcor                  0.041260               0.00000  0.058185   
       scrubbing.5              8.908738               0.00000  0.339532   
       scrubbing.5+gsr          0.079174               0.00000  0.178702   

                        corr_motion_distance  
groups strategy                               
adult  aroma                        0.018173  
       baseline                    -0.108527  
       compcor                     -0.202154  
       scrubbing.5                 -0.118843  
       scrubbing.5+gsr             -0.193409   



FileNotFoundError: [Errno 2] No such file or directory: '/home/seann/scratch/denoise/fmriprep-denoise-benchmark/outputs/denoise-metrics-atlas.5-4.27.25/ds000228/fmriprep-25.0.0/ds000228_fmriprep-25-0-0_desc-stringent_summary.tsv'